In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import itertools
import math

def apr(reg, a, mod, acc):#reg(start, end), 0 <= start < end, 0 <= a < mod, acc >= 1, integers: start, end, acc
    a = a/mod
    i = 2
    bin_str = ""
    size = reg[1] - reg[0]
    j = 1

    while(i < mod*(10**acc) and j <= size):#!acc << test 
        if(a - 1/i >= 0):
            bin_str += "1"
            a -= 1/i
            #circuit.x(reg[1] - j)
        elif(a - 1/i < 0):
            bin_str += "0"
        #else:
            #bin_str += "1"
            #circuit.x(reg[1] - j)
            #break
        i = i << 1
        j += 1
    
    return(bin_str)

def norm_(sigma_, mu_, N):
    """
    Normalization factor for the state. Defined in Eq (7) of paper.
    Inputs:-
    sigma_: standard deviation
    mu_: mean
    N: cutoff the for the infinite sum, i.e. sum_{i=-N}^{i=N} (...)
    """
    return np.sum(np.exp((-(np.arange(-N, N+1, 1) - mu_)**2)/float(sigma_**2)))

def angle_(sigma_, mu_, N=10**3):
    """
    The angle $\alpha$ defined in Eq (12)
    Inputs:-
    sigma_: standard deviation
    mu_: mean
    N: cutoff for the infinite sum in the norm_(..)
    """
    return np.arccos(np.sqrt(norm_(sigma_/2., mu_/2., N)/norm_(sigma_, mu_, N)))

def qubit_strings(n):
    """
    Return a list of n-bit strings in increasing lexicographic order
    Inputs:-
    n: recursion level
    """
    qubit_strings = []
    for q in itertools.product(['0', '1'], repeat=n):
        qubit_strings.append(''.join(q))
    return qubit_strings

def mean_qubit_combo(qub, mu):
    """
    Given an n-bit string, return the corresponding mean for the rotation angle
    at recursion level n
    Inputs:-
    qub: n-bit string
    mu: mean (original)
    """
    mu_out = mu
    for bit in reversed(qub):
        mu_out = (mu_out/2.) - ((1/2.)*int(bit))
    return mu_out

def level_means(mu, n):
    """
    At recursion level n, return a list of all the means used for the various rotation angles
    Inputs:-
    mu: mean (original)
    n: recursion level
    """
    list_mu_out = []
    qb_strings = qubit_strings(n)
    for qb in qb_strings:
        mu_out = mean_qubit_combo(qb, mu)
        list_mu_out.append(mu_out)
    #print(list_mu_out)
    return list_mu_out

def level_angles(sigma, mu, n):
    """
    At recursion level n, return a list of all the rotation angles
    Inputs:-
    sigma: standard deviation (original)
    mu: mean (original)
    n: recursion level
    """
    sigma_out = sigma/(2.**n)
    list_mu = level_means(mu, n)
    # for each (sigma, mu) pair, calculate the corresponding angle
    angles_out = []
    for mu_ in list_mu:
        #print(mu_)
        angles_out.append(angle_(sigma_out, mu_))
    #print(angles_out)
    return angles_out


f1 = open("step_0.txt", 'w')
f2 = open("step_1.txt", 'w')
f3 = open("step_2.txt", 'w')
f1_out = open("step_0_out_a.txt", 'w')
f2_out = open("step_1_out_a.txt", 'w')
f3_out = open("step_2_out_a.txt", 'w')

#1 < mu < 2^N; 1 < sigma < 2^N - mu; mu >= 0 at all step -->
temp0 = []
temp1 = []
temp2 = []
for mu in np.arange(3.0, 8.0, 0.5):
    for sigma in np.arange(1.5, 8-mu, 0.5):
        #print("Step 0")

        ang = level_angles(sigma, mu, 0)
        sigma_apr = apr((0, 4), sigma, 8, 3)
        mu_apr = apr((0, 4), mu, 8, 3)
        ang_apr = apr((0, 5), ang[0], math.pi/2, 3)
        if [sigma_apr, mu_apr] in temp0:
            print((sigma, mu), "repeat", (sigma_apr, mu_apr), ang[0], ang_apr, file=f1)
        else:
            temp0.append([sigma_apr, mu_apr])
            print((sigma, mu), (sigma_apr, mu_apr), ang[0], ang_apr, file=f1)
            print(sigma_apr+mu_apr, 2*ang[0], file=f1_out)

        #print("Step 1")

        ang = level_angles(sigma, mu, 1)

        #ang[0], ang[1] = ang[1], ang[0] #!!!

        sigma_apr = apr((0, 4), sigma, 8, 3)
        mu_apr = apr((0, 4), mu, 8, 3)
        ang_apr = apr((0, 5), ang[0], math.pi/2, 3)
        if [sigma_apr, mu_apr] in temp1:
            print((sigma/2, mu/2), "repeat", (sigma_apr, mu_apr), ang[0], ang_apr, file=f2)
        else:
            temp1.append([sigma_apr, mu_apr])
            print((sigma/2, mu/2), (sigma_apr, mu_apr), ang[0], ang_apr, file=f2)
            print(sigma_apr+mu_apr, 2*ang[0], file=f2_out)

        mu_apr = apr((0, 4), mu-1, 8, 3)
        ang_apr = apr((0, 5), ang[1], math.pi/2, 3)
        if [sigma_apr, mu_apr] in temp1:
            print((sigma/2, (mu-1)/2), "repeat", (sigma_apr, mu_apr), ang[1], ang_apr, file=f2)
        else:
            temp1.append([sigma_apr, mu_apr])
            print((sigma/2, (mu-1)/2), (sigma_apr, mu_apr), ang[1], ang_apr, file=f2)
            print(sigma_apr+mu_apr, 2*ang[1], file=f2_out)

        #print("Step 2")

        ang = level_angles(sigma, mu, 2)

        #ang[1], ang[2] = ang[2], ang[1] #!!!

        sigma_apr = apr((0, 4), sigma, 8, 3)
        mu_apr = apr((0, 4), mu, 8, 3)
        ang_apr = apr((0, 5), ang[0], math.pi/2, 3)
        if [sigma_apr, mu_apr] in temp2:
            print((sigma/4, mu/4), "repeat", (sigma_apr, mu_apr), ang[0], ang_apr, file=f3)
        else:
            temp2.append([sigma_apr, mu_apr])
            print((sigma/4, mu/4), (sigma_apr, mu_apr), ang[0], ang_apr, file=f3)
            print(sigma_apr+mu_apr, 2*ang[0], file=f3_out)

        mu_apr = apr((0, 4), mu-1, 8, 3)
        ang_apr = apr((0, 5), ang[1], math.pi/2, 3)
        if [sigma_apr, mu_apr] in temp2:
            print((sigma/4, (mu/2-1)/2), "repeat", (sigma_apr, mu_apr), ang[1], ang_apr, file=f3)
        else:
            temp2.append([sigma_apr, mu_apr])
            print((sigma/4, (mu/2-1)/2), (sigma_apr, mu_apr), ang[1], ang_apr, file=f3)
            print(sigma_apr+mu_apr, 2*ang[1], file=f3_out)

        mu_apr = apr((0, 4), mu-2, 8, 3)
        ang_apr = apr((0, 5), ang[2], math.pi/2, 3)
        if [sigma_apr, mu_apr] in temp2:
            print((sigma/4, (mu-1)/4), "repeat", (sigma_apr, mu_apr), ang[2], ang_apr, file=f3)
        else:
            temp2.append([sigma_apr, mu_apr])
            print((sigma/4, (mu-1)/4), (sigma_apr, mu_apr), ang[2], ang_apr, file=f3)
            print(sigma_apr+mu_apr, 2*ang[2], file=f3_out)
            
        mu_apr = apr((0, 4), mu-3, 8, 3)
        ang_apr = apr((0, 5), ang[3], math.pi/2, 3)
        if [sigma_apr, mu_apr] in temp2:
            print((sigma/4, ((mu-1)/2 - 1)/2), "repeat", (sigma_apr, mu_apr), ang[3], ang_apr, file=f3)
        else:
            temp2.append([sigma_apr, mu_apr])
            print((sigma/4, ((mu-1)/2 - 1)/2), (sigma_apr, mu_apr), ang[3], ang_apr, file=f3)
            print(sigma_apr+mu_apr, 2*ang[3], file=f3_out)

        #print('\n')

f1.close()
f2.close()
f3.close()
f1_out.close()
f2_out.close()
f3_out.close()
